In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [4]:
import statsmodels.api as sm
# from statsmodels.discrete.discrete_model import Logit
# from statsmodels.discrete.discrete_model import LogitResults
from sklearn.model_selection import train_test_split # X, y, random_state=9
from sklearn.preprocessing import StandardScaler # .fit(X_train), .transform(X) & y

from sklearn.metrics import accuracy_score # (y_test, y_pred)
from sklearn.metrics import classification_report # (y_test, y_pred)
from sklearn.metrics import confusion_matrix # (y_test, y_pred)

In [2]:
dfmodel = pd.read_pickle('../../data/ecommerce/dfmodel_script.pkl', compression='zip')

In [12]:
dfmodel.head(3)
# dfmodel.shape

,timestamp,product_count,transactionid,addtocart,transaction,view,time_delta,time_hour,made_purchase
visitorid,,,,,,,,,
0,2015-09-11 14:55:17,3,NaN,0,0,3,00:05:28,0.091111,0
2,2015-08-07 12:20:57,4,NaN,0,0,8,00:29:13,0.486944,0
6,2015-08-30 21:21:25,3,NaN,1,0,5,21:17:37,21.293611,0


In [ ]:
sm.Logit()
sm.add_constant()

In [11]:
df20 = dfmodel.sample(20000, random_state=46)
df20.head()
df20[df20['transaction'] > 0].shape

(623, 9)

In [13]:
col_all = ['product_count', 'addtocart', 'transaction', 'view', 'time_hour']

In [14]:
def sort_split(df):
    """ return holdout, crossval, tank
    data randomly split among groups for modelling
    2x upsample 11449 purchase rows
    380127 rows main dfmodel
    """
    purmask = df['made_purchase'] == 1
    pur = df[purmask]
    nopur = df[~purmask]
    pur = pur.sample(frac=2., replace=True, random_state=9) # upsample to 22898
    upsamp_rows_size = pur.shape[0]
    # split pur
    holdpur = pur.sample(frac=0.25, random_state=9) #df
    holdmask = pur.index.isin(holdpur.index)
    crosspur = pur[~holdmask] #df
    # split nopur
    times4_size = upsamp_rows_size * 4
    nopur_formodel = nopur.sample(n=times4_size, random_state=9) # pulls 91592
    holdnopur = nopur_formodel.sample(frac=0.25, random_state=9) #df
    holdmaskno = nopur_formodel.index.isin(holdpur.index)
    crossnopur = nopur_formodel[~holdmaskno]  #df
    # set aside tank data unused for model
    tankmask = nopur.index.isin(nopur_formodel.index)
    dftank = nopur[~tankmask]  #df
    # combine for crossval and holdout
    holdpur, holdnopur
    

In [22]:
purmask = dfmodel['made_purchase'] == 1
pur = dfmodel[purmask]
print(dfmodel[~purmask].shape)
print(pur.shape)
print(dfmodel.shape)

(368678, 9)
(11449, 9)
(380127, 9)


In [28]:
testq = pur.sample(frac=0.25, random_state=9)
pur.sample(frac=2., replace=True, random_state=9).shape

(22898, 9)

In [29]:
testq.shape

(2862, 9)

In [31]:
mask = pur.index.isin(testq.index)
mask.shape

(11449,)

In [33]:
pur[~mask].shape

(8587, 9)

In [34]:
22898 * 4

91592